In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime
from random import uniform
import time
import json
import pandas as pd
import sqlite3
import os
from helper import *

In [ ]:
#PATHS TO ELEMENTS

#path to first sites text
eligible_load = '//*[@id="statePods_table"]/tbody/tr[1]/td[1]'

#path to get started button
get_started = '//*[@id="main"]/section/div/div/div/button/div'

#path to enter dob
dob = '//*[@id="relationship"]/div/div[1]/div[1]/div/div[1]/div[1]/input'

#path to sex option
nb = '//*[@id="relationship"]/div/div[1]/div[2]/fieldset/div/div[3]/input'

#path to live option
live = '//*[@id="relationship"]/div/div[2]/div[1]/fieldset/div/div[1]/input'

#path to work option
work = '//*[@id="relationship"]/div/div[2]/div[2]/fieldset/div/div[2]/input'

#path to enter zip
zipcode = '//*[@id="relationship"]/div/div[3]/div/div/div[1]/div[1]/input'

#path to consent option
consent = '//*[@id="acknowledgement"]/div/section/fieldset/div/div/div/input'

#path to submit option
submit = '//*[@id="content"]/section/div[3]/div[2]/button/div'

#path to locate providers button/text
providers = '//*[@id="content"]/section/div[3]/div/button[2]/div'

#path to jb or sb sites
jb = '//*[@id="content"]/section/section[1]/section/section[1]/div[2]/div/p[2]/span/a'
sb = '//*[@id="content"]/section/section[5]/section/section[1]/div[2]/div/p[2]/span/a'

In [ ]:
def load_page():
    search = WebDriverWait(driver, 1).until(
        EC.presence_of_element_located((By.ID, 'OuterDiv'))
    )

    results = driver.find_elements_by_xpath('//*[@id="OuterDiv"]/div')
    slots = int(results[-1].text.split()[-1])
    return slots

In [ ]:
def input_field(path, value):
    
    #wait for presence of element to load
    fill_field = WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.XPATH, path))
    )

    #input value
    fill_field.clear()
    time.sleep(1)
    fill_field.send_keys(value)
    fill_field.send_keys(Keys.RETURN)
    time.sleep(1)

In [ ]:
def click(path):
    
    #wait for presence of element to load
    box = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.XPATH, path))
    )
    
    #click box
    box.click()
    
    time.sleep(1)

In [ ]:
def page_load(path):
    #wait for element to be loaded
    WebDriverWait(driver, 20).until(
        EC.visibility_of_element_located((By.XPATH, path)))

In [ ]:
def fill_out_forms():
    #goes to eligibilty page for ny
    eligible = 'https://am-i-eligible.covid19vaccine.health.ny.gov'
    driver.get(eligible)
        
    
    #waits for text to be loaded
    page_load(eligible_load)
    
    
    #click get started button to go to next page
    try:
        click(get_started)
    except:
        time.sleep(2)
        click(get_started)
    
    
    #fill in dob field
    input_field(dob, '03201935')
    
    
    #click sex option
    click(nb)
    
    
    #click yes for live in ny
    click(live)
    
    
    #click no for work in ny
    click(work)
    
    
    #input zip
    input_field(zipcode, '11554')
    
    
    #click to consent
    try:
        click(consent)
    except:
        time.sleep(2)
        click(consent)

    
    #click to submit
    try:
        click(submit)
    except:
        time.sleep(2)
        click(submit)

    
    #wait for providers page to load
    page_load(providers)
    
    #click to find providers
    click(providers)

    
    #choose site
    try:
        click(jb)
    except:
        time.sleep(2)
        click(jb)
    
    
    #switch to new tab that is opened
    driver.switch_to.window(driver.window_handles[1])

In [ ]:
got_slot = False

# datetime object containing current date and time
now = datetime.now() # current date and time
today = now.strftime("%B %d")

while got_slot == False:
    # path to chromedriver
    PATH = '/Users/Jason/Documents/git/Webdrivers/chromedriver'
    #opens an instance of chrome
    driver = webdriver.Chrome(PATH)
    
    time.sleep(5)
    
    try:
        fill_out_forms()
    except:
        driver.quit()
        continue
    
    for i in range(1000):
        #set variable sleep time
        sleep = uniform(2,4)
        
        #refresh page
        driver.refresh()

        try:
            slots = load_page()
        
        except:
            time.sleep(sleep)
            if i == 999:
                os.system('say "Restart"')
            continue
            
        works = False
        for slot in range(slots):

            try:
                valid = driver.find_element_by_xpath('//*[@id="Marker{}Div"]/div[2]/div/div[2]/div[8]/div/button'.format(slot))
                month = driver.find_element_by_xpath('//*[@id="Marker{}Div"]/div[2]/div/div[2]/div[1]'.format(slot))
                if (valid.text == 'Event Full') or (today in month.text):
                    continue
                elif month.text.split()[1] in ['February', 'March']:
                    valid.click()
                    works = True
                    break
                else:
                    continue
            except:
                continue
    
        if works == False:
            
            time.sleep(sleep)
            if i == 999:
                os.system('say "Restart"')
            continue
        else:
            try:
                tic = '//*[@id="content"]/div[4]/div[2]/div/div/table/tbody/tr[2]/td[3]/input'
                tic_box = WebDriverWait(driver, 3).until(
                    EC.presence_of_element_located((By.XPATH, tic))
                )
                tic_box.click()
    
                next_bttn = driver.find_element_by_xpath('//*[@id="content"]/div[4]/div[2]/div/div/div[2]/fieldset/button[2]')
                next_bttn.click()
                page = True
            except:
                #close current tab
                driver.close()
                
                #switch to original tab
                driver.switch_to.window(driver.window_handles[0])
                
                #choose site
                try:
                    click(jb)
                except:
                    time.sleep(2)
                    click(jb)
                
                #switch to newly opened tab
                driver.switch_to.window(driver.window_handles[1])
                
                page = False
        
        if page:
            os.system('say "Vaccine time."')
            got_slot = True
            break
        elif i == 999:
            os.system('say "Restart"')
            
        time.sleep(sleep) 
    
    if got_slot:
        break
    
    driver.quit()   
    time.sleep(10) 